<a href="https://colab.research.google.com/github/omermehboob/NIDS-using-machine-learning-and-deep-learning/blob/main/DNN_(NIDS_BINARY_7_features).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from __future__ import print_function
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
np.random.seed(1337)  # for reproducibility
from keras.preprocessing import sequence
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Embedding
from keras.utils.np_utils import to_categorical
from sklearn.metrics import (precision_score, recall_score,f1_score, accuracy_score,mean_squared_error,mean_absolute_error)
from sklearn import metrics
from sklearn.preprocessing import Normalizer
import h5py
from keras import callbacks
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, CSVLogger

In [3]:
df = pd.read_csv('/content/drive/MyDrive/Refined_data_set(label_encoding_18_features).csv')

In [4]:
df.shape

(1336498, 20)

In [7]:
df.head()

,Unnamed: 0,Bwd Packet Length Max,Bwd Packet Length Mean,Bwd Packet Length Std,Flow Bytes/s,Flow Duration,Flow IAT Max,Flow IAT Mean,Flow IAT Min,Flow IAT Std,Fwd IAT Total,Fwd Packet Length Max,Fwd Packet Length Mean,Fwd Packet Length Min,Fwd Packet Length Std,Total Backward Packets,Total Fwd Packets,Total Length of Bwd Packets,Total Length of Fwd Packets,Label
0,0,128.0,128.000000,0.00000,2046,193460,164110.0,3.869200e+04,3.0,7.124800e+04,29304.0,35.0,35.00,35.0,0.000000,2,4,256.0,140.0,1
1,1,0.0,0.000000,0.00000,7,4016867,4011885.0,1.004217e+06,10.0,2.005113e+06,4016867.0,6.0,6.00,6.0,0.000000,0,5,0.0,30.0,0
2,2,0.0,0.000000,0.00000,2,5318063,5302748.0,1.772688e+06,97.0,3.057131e+06,5318063.0,6.0,4.00,0.0,3.464102,1,3,0.0,12.0,1
3,3,48.0,48.000000,0.00000,1081081,148,99.0,4.933333e+01,1.0,4.901360e+01,48.0,32.0,32.00,32.0,0.000000,2,2,96.0,64.0,1
4,4,1486.0,186.136364,426.14309,57,117320451,10100000.0,2.550445e+06,39.0,4.191832e+06,117000000.0,453.0,107.64,0.0,181.667939,22,25,4095.0,2691.0,1


In [8]:
newdf = pd.read_csv('/content/drive/MyDrive/Refined_data_set(label_encoding_7_features).csv')

In [9]:
newdf.shape

(1336498, 9)

In [10]:
newdf.head()

,Unnamed: 0,Bwd Packet Length Std,Flow Bytes/s,Total Length of Fwd Packets,Fwd Packet Length Std,Flow IAT Std,Flow IAT Min,Fwd IAT Total,Label
0,0,0.00000,2046,140.0,0.000000,7.124800e+04,3.0,29304.0,1
1,1,0.00000,7,30.0,0.000000,2.005113e+06,10.0,4016867.0,0
2,2,0.00000,2,12.0,3.464102,3.057131e+06,97.0,5318063.0,1
3,3,0.00000,1081081,64.0,0.000000,4.901360e+01,1.0,48.0,1
4,4,426.14309,57,2691.0,181.667939,4.191832e+06,39.0,117000000.0,1


In [12]:
newdf.drop('Unnamed: 0', axis=1, inplace=True)

In [13]:
newdf.head()

,Bwd Packet Length Std,Flow Bytes/s,Total Length of Fwd Packets,Fwd Packet Length Std,Flow IAT Std,Flow IAT Min,Fwd IAT Total,Label
0,0.00000,2046,140.0,0.000000,7.124800e+04,3.0,29304.0,1
1,0.00000,7,30.0,0.000000,2.005113e+06,10.0,4016867.0,0
2,0.00000,2,12.0,3.464102,3.057131e+06,97.0,5318063.0,1
3,0.00000,1081081,64.0,0.000000,4.901360e+01,1.0,48.0,1
4,426.14309,57,2691.0,181.667939,4.191832e+06,39.0,117000000.0,1


In [14]:
X=newdf[["Bwd Packet Length Std", "Flow Bytes/s", "Total Length of Fwd Packets", "Fwd Packet Length Std",
     "Flow IAT Std", "Flow IAT Min", "Fwd IAT Total"]]
y=newdf[["Label"]]

In [15]:
print(X.shape)
print(y.shape)

(1336498, 7)
(1336498, 1)


In [16]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20)

In [17]:
scaler = Normalizer().fit(X_train)
X_train = scaler.transform(X_train)

scaler = Normalizer().fit(y_train)
y_train = scaler.transform(y_train)


In [22]:
model = Sequential()
model.add(Dense(700,input_dim=7,activation='relu'))  
model.add(Dropout(0.10))
model.add(Dense(350,activation='relu'))  
model.add(Dropout(0.10))
model.add(Dense(125,activation='relu'))  
model.add(Dropout(0.10))
model.add(Dense(1,activation='sigmoid'))

model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_8 (Dense)             (None, 700)               5600      
                                                                 
 dropout_6 (Dropout)         (None, 700)               0         
                                                                 
 dense_9 (Dense)             (None, 350)               245350    
                                                                 
 dropout_7 (Dropout)         (None, 350)               0         
                                                                 
 dense_10 (Dense)            (None, 125)               43875     
                                                                 
 dropout_8 (Dropout)         (None, 125)               0         
                                                                 
 dense_11 (Dense)            (None, 1)                

In [23]:
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
model.fit(X_train, y_train, validation_data=(X_test, y_test), batch_size=512, epochs=200)

Epoch 1/200
2089/2089 [==============================] - 9s 4ms/step - loss: 0.3460 - accuracy: 0.8545 - val_loss: 15092216.0000 - val_accuracy: 0.8260
Epoch 2/200
2089/2089 [==============================] - 8s 4ms/step - loss: 0.3291 - accuracy: 0.8603 - val_loss: 9229629.0000 - val_accuracy: 0.8258
Epoch 3/200
2089/2089 [==============================] - 8s 4ms/step - loss: 0.3219 - accuracy: 0.8627 - val_loss: 6117533.0000 - val_accuracy: 0.8280
Epoch 4/200
2089/2089 [==============================] - 9s 4ms/step - loss: 0.2957 - accuracy: 0.8702 - val_loss: 4796417.0000 - val_accuracy: 0.8442
Epoch 5/200
2089/2089 [==============================] - 8s 4ms/step - loss: 0.2427 - accuracy: 0.8985 - val_loss: 3982777.7500 - val_accuracy: 0.8487
Epoch 6/200
2089/2089 [==============================] - 8s 4ms/step - loss: 0.2311 - accuracy: 0.9055 - val_loss: 4136050.2500 - val_accuracy: 0.8433
Epoch 7/200
2089/2089 [==============================] - 8s 4ms/step - loss: 0.2214 - accurac